# 1. Data Preprocessing

## 1.1 Combine Data from different sources
***Collected data from three different datasets including 9846+3512+2775 (16133 in total)dialogs.
Need to merge them into one single file for further processing.***


In [ ]:
import pandas as pd

In [ ]:
df_10K = pd.read_json('/content/drive/MyDrive/DataScience&Machinelearning/Projects/TherapyChatbot/data/Psychology-10K.json')
print(df_10K.shape)#(9846, 3), takes the last two columns only as Q and A

In [ ]:
df_1 = df_10K.iloc[:,1:3]
df_1.columns = ['Question', 'Answer']


In [ ]:
#transform the unicode characters into plain string text
!pip install unidecode
from unidecode import unidecode

In [ ]:
df_combined = pd.read_json('/content/drive/MyDrive/DataScience&Machinelearning/Projects/TherapyChatbot/data/combined_dataset.json',encoding='utf-8')
print(df_combined.shape)#(3512, 2)
df_2 = df_combined
df_2.columns = ['Question', 'Answer']
df_2['Question'].apply(unidecode)
df_2['Answer'].apply(unidecode)


In [ ]:
df_counsel = pd.read_csv('/content/drive/MyDrive/DataScience&Machinelearning/Projects/TherapyChatbot/data/20220401_counsel_chat.csv')
print(df_counsel.shape)#(2775, 10), takes questionTitle+questionText as Q, takes answerText as A
df_counsel.fillna('', inplace=True)# to avoid NaN Merge in case questionTitle or questionText is NaN
df_counsel['Merge'] = df_counsel['questionTitle']+df_counsel['questionText']

In [ ]:
df_3 = df_counsel[['Merge','answerText']]
df_3.columns = ['Question', 'Answer']

df_3.shape

In [ ]:
#saved all the 16133 chats into a csv file
df = pd.concat([df_1, df_2, df_3], axis=0, ignore_index=True)
# df['Client'] = df['input'] + df['Context'] + df['Merge']
# df['Therapist'] = df['output'] + df['Response'] + df['answerText']
# df = df.drop(['input', 'output', 'Context', 'Response', 'Merge', 'answerText'], axis = 1)
print(df.shape)

# use encoding utf-8-sig to handle the byte order mark (BOM), the default encoding for csv file is utf-8
df.to_csv('16133_chat.csv', index=False, encoding='utf-8-sig')
df.to_json('16133_chat.json')

## 1.2 Clean the data
***Clean the data from the following aspects***

*   Remove rows with null values
*   Remove duplicate rows
*   Remove Non-English texts
*   Remove dialogs with too short answers
*   Unnecessary characters such as -------
*   Missing .






In [ ]:
import re

df = pd.read_csv('16133_chat.csv')
print(df.shape)
# Remove rows with null values
df = df.dropna()
print(f'Shape of df after removing rows with null values {df.shape}')

# Remove duplicate rows
df = df.drop_duplicates()
print(f'Shape of df after removing duplicate rows {df.shape}')

# Remove dialogs with too short answers
min_text_length = 100
rows_too_short = df[df['Answer'].str.len() <= min_text_length]
print(rows_too_short)
df = df[df['Answer'].str.len() >= min_text_length]
print(f'Shape of df after removing too short answers {df.shape}')

df['Answer'] = df['Answer'].apply(lambda x: re.sub(r'-+', '', x))


In [ ]:
!pip install langdetect
!pip install nltk
import nltk
nltk.download('punkt')


In [ ]:
from langdetect import detect
from langdetect import detect_langs
from nltk.tokenize import sent_tokenize
import re

# Function to detect the language of a text
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

def delete_non_en(text):
  # Detect the language for each word
  sentences = sent_tokenize(text)
  langs = [detect_language(sentence) for sentence in sentences]
  english_sentences = [sent for sent, lang in zip(sentences, langs) if lang == 'en']
  englist_text = ' '.join(english_sentences)
  return englist_text

# Detect the language for each text in the specified column
df['language'] = df['Answer'].apply(detect_language)
# Remove non-English texts
df.loc[df['language'] != 'en', 'Answer'] = df.loc[df['language'] != 'en', 'Answer'].apply(lambda x: delete_non_en(x))
# rows_non_english = df[df['language'] != 'en']
# print(rows_non_english)
# Drop the 'language' column if desired
df.drop('language', axis=1, inplace=True)



In [ ]:
import re

# Example text with missing periods
text = "This is a sentence without a period This is another sentence without a period"

# Regular expression pattern to detect missing periods
pattern = r'(?<=\w)(?!\s)(?!\.)(?!\?)'

# Find all occurrences of the pattern
missing_periods = re.finditer(pattern, text)

# Print the positions of the missing periods
for match in missing_periods:
    print(f"Missing period at position: {match.start()}")

# Alternatively, you can replace the missing periods with a period
fixed_text = re.sub(pattern, '.', text)
print(fixed_text)

## 1.3 - Train, Validation and Test Split

In [ ]:
train_size = int (0.7 * len(df))
validation_size = int(0.2 * len(df))
shuffled_df = df.sample(frac=1, random_state=42)
train_df = shuffled_df[:train_size]
validation_df = shuffled_df[train_size:(train_size + validation_size)]
test_df = shuffled_df[train_size + validation_size:]
train_df.to_csv('train.csv', index=False)
validation_df.to_csv('validation.csv', index=False)
test_df.to_csv('test.csv', index=False)

train_df.shape
test_df.shape

In [ ]:
print(train_df.shape)
print(test_df.shape)
print(validation_df.shape)

# 2 - Fine-tune the model

# 2.1 -Installation

In [ ]:
![ -d transformers ] || git clone --depth=1 https://github.com/huggingface/transformers

In [ ]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 --quiet
    # torchdata==0.5.1 --quiet

!pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 \
    trl==0.4.4


In [ ]:
!pip install --upgrade transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
import torch
import time
import evaluate
import pandas as pd
import numpy as np

## 2.2 - Load Dataset and LLM

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation':'validation.csv', 'test': 'test.csv'})
dataset

In [ ]:
model_name='google/flan-t5-xxl'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model_name = 'NousResearch/Llama-2-7b-chat-hf'
# model_path = '/content/drive/MyDrive/llama-2-7b'
# original_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_parameters = 0
    all_model_parameters = 0
    for _, param in model.named_parameters():
      all_model_parameters += param.numel()
      if param.requires_grad:
        trainable_model_parameters += param.numel()
    return f"trainable model parameters: {trainable_model_parameters}\nall model parameters:{all_model_parameters}\npercentage of trainable mdoel parameters: {trainable_model_parameters/all_model_parameters}"

print(print_number_of_trainable_model_parameters(original_model))

## 2.3 - Test the model with zero shot inferencing

In [ ]:
index = 205

question = dataset['test'][index]['Question']
answer = dataset['test'][index]['Answer']

prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.

{question}

Reply:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(original_model.generate(inputs['input_ids'], max_new_tokens=200,)[0], skip_special_tokens=True)

dash_line = '_'.join('' for x in range(100))
print(dash_line)
print(f"INPUT PROMPT:\n{prompt}")
print(dash_line)
print(f'REAL THERAPIST REPLY:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION REPLY - ZERO SHOT:\n {output}')

# 3 - Perform full fine_tuning



## 3.1 - Preprocess the Question-Answer Dataset

In [ ]:
from torch.utils.data import random_split

def tokenize_function(example):
    start_prompt = 'If you are a licensed psychologist, please provide this patient with a helpful response to their concern.\n\n'
    end_prompt = '\n\nReply:'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['Question']]
    # print(tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').input_ids.shape)
    example['input_ids'] = pd.Series(tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt')["input_ids"].numpy().tolist())
    example['labels'] = tokenizer(example['Answer'], padding='max_length', truncation=True, return_tensors='pt')["input_ids"]

    return example

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['Question', 'Answer'])
#subsample the dataset
# tokenized_dataset = tokenized_dataset.filter(lambda row, index: index % 10 == 0, with_indices = True)



## 3.2 - Fine-Tune the model with the tokenized dataset

In [ ]:
fine_tune_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)


In [ ]:
output_dir = f'./drive/MyDrive/aitherapy/therapist-ai-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-4,
    num_train_epochs=10,
    weight_decay=0.02,
    logging_steps=100,
    max_steps=2000
)

trainer = Trainer(
    model=fine_tune_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation']
)

In [ ]:
trainer.train()

## 3.3 - Evaluate the model qualitatively

In [ ]:
index = 205

question = dataset['test'][index]['Question']
answer = dataset['test'][index]['Answer']

prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.
{question}

Reply:
"""

inputs = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')
model_output = fine_tune_model.generate(inputs, max_new_tokens=200, num_beams=1)
model_text_output = tokenizer.decode(model_output[0], skip_special_tokens=True)


dash_line = '_'.join('' for x in range(100))
print(dash_line)
print(f"INPUT PROMPT:\n{prompt}")
print(dash_line)
print(f'REAL THERAPIST REPLY:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION REPLY - Fine Tuned:\n {model_text_output}')

## 3.4 - Evaluate the model quantitatively (with ROUGE metric)

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
questions = dataset['test'][0:10]['Question']
real_therapy_replies = dataset['test'][0:10]['Answer']

model_replies = []

for _, question in enumerate(questions):
  prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.
  {question}
  Reply:"""
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')

  model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  model_text_outputs = tokenizer.decode(model_outputs[0], skip_special_tokens=True)
  model_replies.append(model_text_outputs)

zipped_summaries = list(zip(real_therapy_replies, model_replies))

df = pd.DataFrame(zipped_summaries, columns = ['real_therapy_replies', 'model_replies'])

model_results = rouge.compute(
    predictions=model_replies,
    references=real_therapy_replies[0:len(model_replies)],
    use_aggregator=True,
    use_stemmer=True,
)

print('model rouge:')
print(model_results)

# 4 - Perform Parameter Efficient Fine-Tuning (PEFT)

## 4.1 - Setup the PEFT/LoRA model for fine-tuning

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=['q','v'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
original_peft_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
peft_model = get_peft_model(original_peft_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

## 4.2 - Train PEFT Adapter

In [ ]:
output_dir = f'./peft-ai-therapist-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_steps=100,
    max_steps=2000
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset['train'],
)

In [ ]:
peft_trainer.train()

peft_model_path = './peft_ai_therapist_checkpoint_local'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft_ai_therapist_checkpoint_local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False).to('cuda')
print(print_number_of_trainable_model_parameters(peft_model))

## 4.3 - Evaluate the model qualitatively

In [ ]:
index = 100

question = dataset['test'][index]['Question']
answer = dataset['test'][index]['Answer']

prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.

{question}

Reply:
"""

inputs = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')

original_model_outputs = original_model.generate(inputs, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(inputs=inputs, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '_'.join('' for x in range(100))
print(dash_line)
print(f"INPUT PROMPT:\n{prompt}")
print(dash_line)
print(f'REAL THERAPIST REPLY:\n{answer}\n')
print(dash_line)
print(f'Original MODEL GENERATION REPLY - Fine Tuned:\n {original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL GENERATION REPLY - Fine Tuned:\n {peft_model_text_output}')

## 4.4 - Evaluate the model quantitatively (ROUGE)

In [ ]:
questions = dataset['test'][0:10]['Question']
real_therapy_replies = dataset['test'][0:10]['Answer']

original_model_replies = []
peft_model_replies = []

for _, question in enumerate(questions):
  prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.
  {question}
  Reply:"""
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  original_model_outputs = original_model.generate(input_ids=inputs, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

  peft_model_outputs = peft_model.generate(input_ids=inputs, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

  original_model_replies.append(original_model_text_output)
  peft_model_replies.append(peft_model_text_output)

zipped_summaries = list(zip(real_therapy_replies, original_model_replies, peft_model_replies))

df = pd.DataFrame(zipped_summaries, columns = ['real_therapy_replies', 'original_model_replies', 'peft_model_replies'])

original_model_results = rouge.compute(
    predictions=original_model_replies,
    references=real_therapy_replies[0:len(original_model_replies)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_replies,
    references=real_therapy_replies[0:len(peft_model_replies)],
    use_aggregator=True,
    use_stemmer=True,
)
print('original model rouge:')
print(original_model_results)
print('peft model rouge:')
print(peft_model_results)


# 5 - Fine-Tune FLAN-T5 with reinforcement Learning (PPO) and PEFT to Generate More-Empathetic Relies

## 5.1 - Set up Kernel and Required Dependencies

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

#trl: transformer reinforcement learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

# tqdm library makes the loops show a smart progress meter
from tqdm import tqdm
tqdm.pandas()

## 5.2 - Prepare Reward Model and Empathy Evaluator

### 5.2.1 - Load Data and PPO model

In [ ]:
def tokenize(example):
  prompt = f"""
If you are a licensed psychologist, please provide this patient with a helpful response to their concern.

{question}

Reply:
"""
  example['input_ids'] = tokenizer.encode(prompt)
  example['query'] = tokenizer.decode(example['input_ids'])
  return example

# dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation':'validation.csv', 'test': 'test.csv'})
ppo_dataset = dataset.map(tokenize, batched=False)
ppo_dataset.set_format(type='torch')


In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model, torch_type=torch.bfloat16, is_trainable=True)
print(f'PPO model parameters to be update:\n {print_number_of_trainable_model_parameters(ppo_model)}\n')#valuehead + 769
print(ppo_model.v_head)

In [ ]:
ref_model = create_reference_model(ppo_model)
print(f'reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')


### 5.2.2 - Prepare Reward Model

In [1]:
empathy_model_name = 'facebook/roberta-hate-speech-dynabench-r4-target'
empathy_tokenizer = AutoTokenizer.from_pretrained(empathy_model_name, device_map='auto')
empathy_model = AutoModelForSequenceClassification.from_pretrained(empathy_model_name, device_map='auto')
print(empathy_model.config.id2label)

NameError: ignored

In [ ]:
empathy_text = 'That is not your fault at all'

empathy_input_ids = empathy_tokenizer(empathy_text, return_tensors='pt').input_ids.to('cuda')

logits = empathy_model(input_ids=empathy_input_ids).logits
print(f'ligits [empathic, not empathic]:{logits.tolist()[0]}')

probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [empathic, not empathic]: {probabilities}')

empathy_index = 0
empathy_reward = (logits[:,empathy_index]).tolist()
print(f'reward (low): {empathy_reward}')

In [ ]:
non_empathy_text = 'That is all your fault'

empathy_input_ids = empathy_tokenizer(non_empathy_text, return_tensors='pt').input_ids.to('cuda')

logits = empathy_model(input_ids=empathy_input_ids).logits
print(f'ligits [empathic, not empathic]:{logits.tolist()[0]}')

probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [empathic, not empathic]: {probabilities}')

empathy_index = 0
empathy_reward = (logits[:,empathy_index]).tolist()
print(f'reward (high): {empathy_reward}')

In [ ]:
device = 0 if torch.cuda.is_available() else 'cpu'

sentiment_pipe = pipeline('sentiment-analysis',
                          model=empathy_model_name,
                          device=device)
reward_logits_kwargs = {
    'top_k': None,
    'function_to_apply':'none',
    'batch_size':16
}

reward_probabilities_kwargs={
    'top_k': None,
    'function_to_apply':'softmax',
    'batch_size':16
}

print('Reward model output for empathic text:')
print(sentiment_pipe(empathy_text, **reward_logits_kwargs))
print(sentiment_pipe(empathy_text, **reward_probabilities_kwargs))
print('Reward model output for non-empathic text:')
print(sentiment_pipe(non_empathy_text, **reward_logits_kwargs))
print(sentiment_pipe(non_empathy_text, **reward_probabilities_kwargs))

### 5.2.3 - Evaluate Empathy

In [ ]:
empathy_evaluator = evaluate.load('toxicity',
                                  empathy_model_name,
                                  module_type='measurement',
                                  toxic_label='hate')

In [ ]:
empathy_score = empathy_evaluator.compute(predictions=[non_empathy_text])
print('Empathy score for non-empathy text: ')
print(empathy_score['toxicity'])

empathy_score = empathy_evaluator.compute(predictions=[empathy_text])
print('Empathy score for empathy text:')
print(empathy_score['toxicity'])

In [ ]:
def evaluate_empathy(model,
                     empathy_evaluator,
                     tokenizer,
                     dataset,
                     num_samples):
  max_new_tokens = 200
  empathies = []
  input_texts = []
  for i, sample in tqdm(enumerate(dataset)):
    input_text = sample['query']

    if i > num_samples:
      break

    input_ids = tokenizer(input_text, return_tensors='pt', padding=True).input_ids.to('cuda')

    generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                         tok_k=0.0,
                                         top_p=1.0,
                                         do_sample=True)

    response_token_ids = model.generate(input_ids=input_ids,
                                        generation_config=generation_config)

    generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

    empathy_score = empathy_evaluator.compute(predictions=[input_text + ' ' + generated_text])

    empathies.extend(empathy_score['toxicity'])

  mean = np.mean(empathies)
  std = np.std(empathies)

  return mean, std

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='cuda')
mean_before_detoxification, std_before_detoxification = evaluate_empathy(model=ref_model,
                                                                         empathy_evaluator=empathy_evaluator,
                                                                         tokenizer=tokenizer,
                                                                         dataset=ppo_dataset['test'],
                                                                         num_samples=10)

print(f'empathy [mean, std] before detox: [{mean_before_detoxification, std_before_detoxification}]')

## 5.3 - Perform Fine-Tuning to Empathize the Replies

### 5.3.1 - Initialize PPOTrainer

In [ ]:
learning_rate = 1.41e-5
max_ppo_epochs = 10
mini_batch_size = 4
batch_size = 16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         tokenizer=tokenizer,
                         dataset=ppo_dataset['train'],
                         data_collator=collator)

### 5.3.2 - Fine-tune the model

In [ ]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    'min_length': 5,
    'top_k': 0.0,
    "top_p": 1.0,
    'do_sample': True
}

reward_kwargs = {
    'top_k': None,
    'function_to_apply': 'none',
    'batch_size': 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
  if step > max_ppo_steps:
    break

  prompt_tensors = batch['input_ids']
  reply_tensors = []

  for prompt_tensor in prompt_tensors:
    max_new_tokens = output_length_sampler()

    generation_kwargs['max_new_tokens'] = max_new_tokens
    reply = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

    reply_tensors.append(reply.squeeze()[-max_new_tokens:])

  batch['response'] = [tokenizer.decode(r.squeeze()) for r in reply_tensors]

  query_response_pairs = [q + r for q, r in zip(batch['query'], batch['response'])]
  rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

  reward_tensors = [torch.tensor(reward[empathy_index]['score']) for reward in rewards]

  stats = ppo_trainer.step(prompt_tensors, reply_tensors, reward_tensors)
  ppo_trainer.log_stats(stats, batch, reward_tensors)

  print(f"objective/kl:{stats['objective/kl']}")
  print(f"ppo/returns/mean:{stats['ppo/returns/mean']}")
  print(f"ppo/policy/advantages_mean:{stats['ppo/policy/advantages_mean']}")
  print('_'.join('' for x in range(100)))

### 5.3.3 - Evaluate the model qualitatively

In [ ]:
batch_size = 20
compare_results = {}

df_batch = ppo_dataset['test'][0:batch_size]

compare_results['query'] = df_batch['query']
prompt_tensors = df_batch['input_ids']

reply_tensors_ref = []
reply_tensors = []

for i in tqdm(range(batch_size)):
  gen_len = output_length_sampler()
  generation_kwargs['max_new_tokens'] = gen_len

  reply = ref_model.generate(input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
                                                       **generation_kwargs).squeeze()[-gen_len:]
  reply_tensors_ref.append(reply)
  reply = ppo_model.generate(input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
                                                       **generation_kwargs).squeeze()[-gen_len:]
  reply_tensors.append(reply)

compare_results['response_before'] = [tokenizer.decode(reply_tensors_ref[i]) for i in range(batch_size)]
compare_results['response_after'] = [tokenizer.decode(reply_tensors[i]) for i in range(batch_size)]

texts_before = [d + s for d, s in zip(compare_results['query'], compare_results['response_before'])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results['reward_before'] = [reward[empathy_index]['score'] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results['query'], compare_results['response_after'])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results['reward_after'] = [reward[empathy_index]['score'] for reward in rewards_after]

In [ ]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results['reward_diff'] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)

In [ ]:
df_compare_results_sorted